In [1]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import time

# Preprocesamiento

In [2]:
df = pd.read_csv('../Data/df.csv')
df.head(3)

,Prior_purchases_/_Customer_care_calls,Prior_purchases_/_Customer_rating,Cost_of_the_Product_*_Customer_rating,Customer_rating_/_Customer_care_calls,Discount_offered_%_Prior_purchases,Weight_in_gms_%_Customer_rating,Cost_of_the_Product_%_Discount_offered,Customer_rating_/_Cost_of_the_Product,Prior_purchases_+_Weight_in_gms,Prior_purchases_/_Cost_of_the_Product,...,Discount_offered_/_1,Discount_offered_-_Prior_purchases,Customer_rating_-_Prior_purchases,Cost_of_the_Product_*_Customer_care_calls,PERCENTILE(Cost_of_the_Product),Customer_care_calls_/_Discount_offered,Cost_of_the_Product_-_Customer_care_calls,Discount_offered_/_Customer_care_calls,Weight_in_gms,Reached_on_Time_Y_N
0,0.75,1.50,354.00,0.50,2.00,1.00,1.00,0.01,1236.00,0.02,...,44.00,41.00,-1.00,708.00,0.29,0.09,173.00,11.00,1233,1
1,0.50,0.40,1080.00,1.25,1.00,3.00,39.00,0.02,3090.00,0.01,...,59.00,57.00,3.00,864.00,0.51,0.07,212.00,14.75,3088,1
2,2.00,2.00,366.00,1.00,0.00,0.00,39.00,0.01,3378.00,0.02,...,48.00,44.00,-2.00,366.00,0.33,0.04,181.00,24.00,3374,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 100 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Prior_purchases_/_Customer_care_calls      10999 non-null  float64
 1   Prior_purchases_/_Customer_rating          10999 non-null  float64
 2   Cost_of_the_Product_*_Customer_rating      10999 non-null  float64
 3   Customer_rating_/_Customer_care_calls      10999 non-null  float64
 4   Discount_offered_%_Prior_purchases         10999 non-null  float64
 5   Weight_in_gms_%_Customer_rating            10999 non-null  float64
 6   Cost_of_the_Product_%_Discount_offered     10999 non-null  float64
 7   Customer_rating_/_Cost_of_the_Product      10999 non-null  float64
 8   Prior_purchases_+_Weight_in_gms            10999 non-null  float64
 9   Prior_purchases_/_Cost_of_the_Product      10999 non-null  float64
 10  Discount_offered_*_We

In [4]:
df['Reached_on_Time_Y_N'].value_counts()

Reached_on_Time_Y_N
1    6563
0    4436
Name: count, dtype: int64

Hay desbalance de clases, primero se entrena un modelo base y luego se realizan experimentos de sampling.

In [5]:
X = df.drop(columns=['Reached_on_Time_Y_N'])
y = df['Reached_on_Time_Y_N']

# FLAML

In [6]:
RANDOM_STATE = 22

In [7]:
from flaml import AutoML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Entrenamiento
automl = AutoML(random_state=RANDOM_STATE)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

start_train_flaml = time.time()
automl.fit(X_train, y_train, task="classification", time_budget=60, verbose=0, seed=RANDOM_STATE)
end_train_flaml = time.time()

train_time_flaml = end_train_flaml - start_train_flaml
print(f"Tiempo de entrenamiento: {train_time_flaml:.4f} segundos")

# Predicciones
start_pred_flaml = time.time()
y_pred = automl.predict(X_test)
end_pred_flaml = time.time()
predict_time_flaml = end_pred_flaml - start_pred_flaml
print(f"Tiempo de predicción: {predict_time_flaml:.4f} segundos")

# Probabilidades (opcional, si tu modelo las soporta)
y_proba = automl.predict_proba(X_test)

# Métricas principales
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1-score: {f1:.4f}")

# Reporte más completo
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

# Mejor modelo encontrado
print("\n--- Mejor modelo ---")
print(automl.best_estimator)
#print(automl.best_model)
print(automl.best_config)
print(f"Best accuracy (cv): {automl.best_loss:.4f}")


Tiempo de entrenamiento: 60.0690 segundos
Tiempo de predicción: 0.0165 segundos
✅ Accuracy: 0.6900
✅ Precision: 0.7696
✅ Recall: 0.6900
✅ F1-score: 0.6865

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.57      0.91      0.70       887
           1       0.90      0.54      0.67      1313

    accuracy                           0.69      2200
   macro avg       0.74      0.73      0.69      2200
weighted avg       0.77      0.69      0.69      2200


--- Confusion Matrix ---
[[811  76]
 [606 707]]

--- Mejor modelo ---
lgbm
{'n_estimators': 14, 'num_leaves': 8, 'min_child_samples': 7, 'learning_rate': 0.13469138604865435, 'log_max_bin': 10, 'colsample_bytree': 0.8164506244638708, 'reg_alpha': 0.0009765625, 'reg_lambda': 20.766656285320355}
Best accuracy (cv): 0.2187


Se realizan experimentos de undersampling y oversampling, para determinar la mejor forma de resolver el desbalance de clases.

In [8]:
from flaml import AutoML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

def evaluate_sampling(X_train, y_train, X_test, y_test, sampler_name, sampler):
    # Aplicar sampling
    X_res, y_res = sampler.fit_resample(X_train, y_train)
    print(f"\n🔹 {sampler_name}: distribución original {Counter(y_train)} → nueva {Counter(y_res)}")

    # Entrenar modelo
    automl = AutoML()
    start_train = time.time()
    automl.fit(X_res, y_res, task="classification", time_budget=60, verbose=0, seed=RANDOM_STATE)
    end_train = time.time()
    train_time = end_train - start_train
    print(f"Tiempo de entrenamiento: {train_time:.4f} segundos")

    # Predicciones
    start_pred = time.time()
    y_pred = automl.predict(X_test)
    end_pred = time.time()
    predict_time = end_pred - start_pred
    print(f"Tiempo de predicción: {predict_time:.4f} segundos")

    # Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"✅ Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))
    print(confusion_matrix(y_test, y_pred))
    print(f"Mejor modelo: {automl.best_estimator}")

    return automl


# --- Random UnderSampling ---
rus = RandomUnderSampler(random_state=42)
automl_rus = evaluate_sampling(X_train, y_train, X_test, y_test, "RandomUnderSampler", rus)

# --- Random OverSampling ---
ros = RandomOverSampler(random_state=42)
automl_ros =evaluate_sampling(X_train, y_train, X_test, y_test, "RandomOverSampler", ros)



🔹 RandomUnderSampler: distribución original Counter({1: 5250, 0: 3549}) → nueva Counter({0: 3549, 1: 3549})
Tiempo de entrenamiento: 60.1312 segundos
Tiempo de predicción: 0.0340 segundos
✅ Accuracy: 0.6859 | Precision: 0.8145 | Recall: 0.6859 | F1: 0.6751
              precision    recall  f1-score   support

           0       0.56      0.99      0.72       887
           1       0.98      0.48      0.65      1313

    accuracy                           0.69      2200
   macro avg       0.77      0.74      0.68      2200
weighted avg       0.81      0.69      0.68      2200

[[877  10]
 [681 632]]
Mejor modelo: extra_tree

🔹 RandomOverSampler: distribución original Counter({1: 5250, 0: 3549}) → nueva Counter({1: 5250, 0: 5250})
Tiempo de entrenamiento: 67.6409 segundos
Tiempo de predicción: 0.0255 segundos
✅ Accuracy: 0.6523 | Precision: 0.6692 | Recall: 0.6523 | F1: 0.6555
              precision    recall  f1-score   support

           0       0.56      0.68      0.61       887
 

Los conjuntos con el experimento de undersampling tuvieron mejores resultados, por lo tanto, se utilizarán para posteriores experimentos con otros modelos de AutoML.

In [9]:
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [11]:
# Exportar conjunto de test para notebook de interpretación
X_test.to_csv('../Data/Test/X_test.csv', index=True)
y_test.to_csv('../Data/Test/y_test.csv', index=True)

In [31]:
# Exportar mejor modelo FLAML como pickle
import joblib

flaml_mejor_nombre = automl_rus.best_estimator 
print(flaml_mejor_nombre)
flaml_mejor_modelo = automl_rus.model
joblib.dump(flaml_mejor_modelo, '../Models/flaml_extra_tree.joblib')

extra_tree


['../Models/flaml_extra_tree.joblib']

El mejor modelo fue exportado para importar posteriormente en la notebook de Interpretación.

# LazyClassifier

In [16]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None, random_state=RANDOM_STATE)
start_train_lc = time.time()

models, predictions = clf.fit(X_train_rus, X_test, y_train_rus, y_test)
end_train_lc = time.time()
train_time_lc = end_train_lc - start_train_lc
print(f"Tiempo de entrenamiento: {train_time_lc:.4f} segundos")
models

  0%|          | 0/29 [00:00<?, ?it/s]

Tiempo de entrenamiento: 49.5157 segundos


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.69,0.73,0.73,0.68,4.34
LogisticRegression,0.68,0.73,0.73,0.67,0.44
LinearSVC,0.68,0.73,0.73,0.67,2.39
CalibratedClassifierCV,0.67,0.72,0.72,0.66,3.93
SVC,0.67,0.72,0.72,0.66,5.80
RidgeClassifierCV,0.68,0.72,0.72,0.67,0.56
RidgeClassifier,0.68,0.72,0.72,0.67,0.19
LinearDiscriminantAnalysis,0.68,0.72,0.72,0.67,0.60
LGBMClassifier,0.68,0.72,0.72,0.67,0.97


In [28]:
best_model_name = predictions.index[0]
best_model = clf.models[best_model_name]

start_pred_lc = time.time()
y_pred = best_model.predict(X_test)
end_pred_lc = time.time()
predict_time_lc = end_pred_lc - start_pred_lc

print(f"Modelo seleccionado: {best_model_name}")
print(f"Tiempo de predicción: {predict_time_lc:.4f} segundos")

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")

Modelo seleccionado: AdaBoostClassifier
Tiempo de predicción: 0.0570 segundos
Accuracy: 0.6859


In [38]:
joblib.dump(best_model, '../Models/lazyclassifier_ada_boost_classifier.joblib')

['../Models/lazyclassifier_ada_boost_classifier.joblib']

# Pycaret

In [18]:
from pycaret.classification import *
import time

TARGET = "Reached_on_Time_Y_N"

df_train = X_train_rus.copy()
df_train[TARGET] = y_train_rus.values if hasattr(y_train_rus, "values") else y_train_rus
df_test = X_test.copy()
df_test[TARGET] = y_test.values if hasattr(y_test, "values") else y_test

# --- Setup ---
_ = setup(
    data=df_train,
    target=TARGET,
    session_id=RANDOM_STATE,
    test_data=df_test,       
    fold=5,
    verbose=False,
    imputation_type="simple",
)

# --- TIEMPO DE ENTRENAMIENTO (AutoML completo) ---
start_pycaret = time.perf_counter()
best = compare_models()  # entrena/evalúa varios modelos con CV y elige el mejor
end_pycaret = time.perf_counter()

pycaret_time = end_pycaret - start_pycaret
print(f"[AutoML] Tiempo total compare_models: {pycaret_time:.4f} s")

# --- Leaderboard ---
lb = pull()
print(lb.head(10))

# --- Entrenar solo el mejor ---
start_finalize = time.perf_counter()
best_final = finalize_model(best)  # reentrena el mejor con todo el train
end_finalize = time.perf_counter()
finalize_time = end_finalize - start_finalize
print(f"[Best Model] Tiempo de entrenamiento final (finalize_model): {finalize_time:.4f} s")

# --- TIEMPO DE PREDICCIÓN ---
X_test_pc = get_config("X_test")

# Predicción completa con pipeline (incluye transformaciones)
start_pred = time.perf_counter()
pred_df = predict_model(best_final, data=X_test_pc, raw_score=False)
end_pred = time.perf_counter()
predict_time = end_pred - start_pred
print(f"[Best Model] Tiempo de predicción (predict_model): {predict_time:.6f} s")
print(f"[Best Model] Latencia promedio por muestra: {predict_time/len(X_test_pc):.8f} s/row")

# Solo .predict() del pipeline (sin columnas extra)
start_pred2 = time.perf_counter()
_ = best_final.predict(X_test_pc)
end_pred2 = time.perf_counter()
predict_time2 = end_pred2 - start_pred2
print(f"[Best Model] Tiempo de .predict(): {predict_time2:.6f} s")
print(f"[Best Model] Latencia promedio por muestra (.predict): {predict_time2/len(X_test_pc):.8f} s/row")


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7226,0.7287,0.4908,0.9150,0.6388,0.4452,0.5025,3.9100
ada,Ada Boost Classifier,0.7158,0.7293,0.4999,0.8804,0.6375,0.4317,0.4788,0.8840
qda,Quadratic Discriminant Analysis,0.7087,0.7199,0.4508,0.9324,0.6069,0.4173,0.4877,0.1000
lda,Linear Discriminant Analysis,0.7072,0.7270,0.4886,0.8690,0.6251,0.4145,0.4612,0.0800
ridge,Ridge Classifier,0.7046,0.7258,0.4813,0.8707,0.6193,0.4091,0.4577,0.0420
lightgbm,Light Gradient Boosting Machine,0.7020,0.7306,0.5334,0.8053,0.6416,0.4041,0.4294,0.6300
rf,Random Forest Classifier,0.7015,0.7344,0.5376,0.8000,0.6429,0.4029,0.4267,1.0940
nb,Naive Bayes,0.6972,0.7221,0.4334,0.9178,0.5886,0.3945,0.4645,0.0500
et,Extra Trees Classifier,0.6965,0.7332,0.5500,0.7779,0.6444,0.3931,0.4111,0.3920
lr,Logistic Regression,0.6915,0.7186,0.4942,0.8162,0.6156,0.3829,0.4168,2.5580


[AutoML] Tiempo total compare_models: 79.7087 s
                                    Model  Accuracy  AUC  Recall  Prec.   F1  \
gbc          Gradient Boosting Classifier      0.72 0.73    0.49   0.92 0.64   
ada                  Ada Boost Classifier      0.72 0.73    0.50   0.88 0.64   
qda       Quadratic Discriminant Analysis      0.71 0.72    0.45   0.93 0.61   
lda          Linear Discriminant Analysis      0.71 0.73    0.49   0.87 0.63   
ridge                    Ridge Classifier      0.70 0.73    0.48   0.87 0.62   
lightgbm  Light Gradient Boosting Machine      0.70 0.73    0.53   0.81 0.64   
rf               Random Forest Classifier      0.70 0.73    0.54   0.80 0.64   
nb                            Naive Bayes      0.70 0.72    0.43   0.92 0.59   
et                 Extra Trees Classifier      0.70 0.73    0.55   0.78 0.64   
lr                    Logistic Regression      0.69 0.72    0.49   0.82 0.62   

          Kappa  MCC  TT (Sec)  
gbc        0.45 0.50      3.91  
ada  

[Best Model] Tiempo de predicción (predict_model): 0.133801 s
[Best Model] Latencia promedio por muestra: 0.00006082 s/row
[Best Model] Tiempo de .predict(): 0.013221 s
[Best Model] Latencia promedio por muestra (.predict): 0.00000601 s/row


In [35]:
best_model_pycaret = str(best)
best_model_pycaret

"GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,\n                           learning_rate=0.1, loss='log_loss', max_depth=3,\n                           max_features=None, max_leaf_nodes=None,\n                           min_impurity_decrease=0.0, min_samples_leaf=1,\n                           min_samples_split=2, min_weight_fraction_leaf=0.0,\n                           n_estimators=100, n_iter_no_change=None,\n                           random_state=22, subsample=1.0, tol=0.0001,\n                           validation_fraction=0.1, verbose=0,\n                           warm_start=False)"

In [37]:
joblib.dump(best_final, '../Models/pycaret_gradient_boosting_classifier.joblib')

['../Models/pycaret_gradient_boosting_classifier.joblib']